In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from skimage import feature
from PIL import Image, ImageFont, ImageDraw
import pandas as pd
from matplotlib import cm
import rasterio
import os
import imageio
import os


# Functions:
## Landsat 8
l8_water_mask_NDVI(): combined approach of LSWI and NDVI for rice field detection.

INDICE_Timeseries(): creates the time series plots of the used indices over the entire available time period. Calculates the mean value of the raster. 

In [ ]:
def norm(band):
    band_min, band_max = band.min(), band.max()
    return ((band - band_min)/(band_max - band_min))

In [ ]:
def scale(x):
    return((x - np.nanpercentile(x, 2)) / (np.nanpercentile(x, 98) - np.nanpercentile(x, 2)))

In [ ]:
def show_RGB(file):
    ds = rasterio.open(f"S:\course\geo441\stud\mfeyen\data\landsat 8\{file}")

    b1 = ds.read(4)
    b1 = b1[2190:2473, 1093:1475]
    b1 = norm(b1).astype(np.float)
    b1 = scale(b1)
        
    b2 = ds.read(3)
    b2 = b2[2190:2473, 1093:1475]
    b2 = norm(b2).astype(np.float)
    b2 = scale(b2)

    b3 = ds.read(2)
    b3 = b3[2190:2473, 1093:1475]
    b3 = norm(b3).astype(np.float)
    b3 = scale(b3)

    stack = np.dstack((b1, b2, b3))
    return stack

In [ ]:
def l8_water_mask_NDVI(file):
    ds = rasterio.open(f"S:\course\geo441\stud\mfeyen\data\landsat 8\{file}")
    
    # LSWI (NIR (Band5) & SWIR2 (Band7))
    b5 = ds.read(5)
    b5 = b5[2190:2473, 1093:1475]
    
    b7 = ds.read(7)
    b7 = b7[2190:2473, 1093:1475]
  
    #np.seterr(divide='ignore', invalid='ignore')

    lswi = (b5.astype(float) - b7.astype(float))/(b5 + b7)
    
    
    # NDVI
    b5 = ds.read(5)
    b5 = b5[2190:2473, 1093:1475]
        
    b4 = ds.read(4)
    b4 = b4[2190:2473, 1093:1475]
              
    ndvi = (b5.astype(float) - b4.astype(float)) / (b5 + b4)
    
    v = lswi - ndvi + 0.05
    
    water_mask = np.zeros((283, 382))

    for i in range(len(v)):
        for j in range(len(v[i])):
            if v[i][j] > 0:
                water_mask[i][j] = 1
            else: 
                water_mask[i][j] =  float("nan")
            #ndmi_mask[i][j] = 0

            
    return water_mask

In [ ]:
import os
path = "S:\course\geo441\stud\mfeyen\data\landsat 8"
dir_list = os.listdir(path)

In [ ]:
def ndwi_Timeseries(input_list):
    list_values = []
    for filename in input_list:
        ds = rio.open(f"{path}/{filename}")
        b3 = ds.read(3)
        b3 = b3[2190:2473, 1093:1475]
       
        b5 = ds.read(5)
        b5 = b5[2190:2473, 1093:1475]
        
        ndwi = (b3.astype(float) - b5.astype(float)) / (b3 + b5)

        stats = np.nanmean(ndwi)
        list_values.append(stats)
    return list_values

In [ ]:
def ndmi_Timeseries(input_list):
    list_values = []
    for filename in input_list:
        ds = rio.open(f"{path}/{filename}")
        
        b5 = ds.read(5)
        b5 = b5[2190:2473, 1093:1475]
               
        b6 = ds.read(6)
        b6 = b6[2190:2473, 1093:1475]
        
        ndmi = (b5.astype(float) - b6.astype(float)) / (b5 + b6)
        stats = np.nanmean(ndmi)
        list_values.append(stats)
    return list_values

In [ ]:
def ndvi_Timeseries(input_list):
    list_values = []
    for filename in input_list:
        ds = rio.open(f"{path}/{filename}")
        
        b5 = ds.read(5)
        b5 = b5[2190:2473, 1093:1475]
        #b5 = norm(b5).astype(np.float)
        #b5 = scale(b5)
        
        b4 = ds.read(4)
        b4 = b4[2190:2473, 1093:1475]
        #b6 = norm(b6).astype(np.float)
        #b6 = scale(b6)
        
        ndvi = (b5.astype(float) - b4.astype(float)) / (b5 + b4)
        #stats = zonal_stats(ndmi, ndmi, affine = affine, stats=['min', 'max', 'mean', 'median', 'majority'])
        stats = np.nanmean(ndvi)
        list_values.append(stats)
    return list_values
        
#test_ndvi = ndvi_Timeseries(dir_list)

## SAR
SAR_Timeseries(): Calculates the mean backscatter value in dB. 

sar_time_series_image(): For a certain timeframe, the water mask is calculated and displayed on its raw image.

In [ ]:
def sar_water_mask(file):
    img = rasterio.open(f"S:\course\geo441\stud\mfeyen\data\S1_vh\{file}")
    band1 = img.read(1)
    band1 = band1[2190:2473, 1093:1475]
    band1 = np.log10(band1)
    
    water_mask = np.zeros((283, 382))
    
    for i in range(len(band1)):
        for j in range(len(band1[i])):
            if band1[i][j] < -2:
                water_mask[i][j] = 1
            else: water_mask[i][j] = float("nan")
    return water_mask

In [ ]:
path_SAR = "S:\course\geo441\stud\mfeyen\data\S1_vh"
dir_list_SAR = os.listdir(path_SAR)

In [ ]:
name_list_SAR = []
for filename in dir_list_SAR:
    name_list_SAR.append(filename[12:20])

In [ ]:
def read_sar(imput):
    img = rio.open(f"{path_SAR}/{imput}")
    band1 = img.read(1)
    band1 = band1[2190:2473, 1093:1475]
    band1 = np.log10(band1)
    return band1

In [ ]:
def SAR_Timeseries(input_list):
    list_values = []
    for filename in input_list:
        img = rio.open(f"{path_SAR}/{filename}")
        band1 = img.read(1)
        band1 = band1[2190:2473, 1093:1475]
        band1 = np.log10(band1)

        stats = np.nanmean(band1)
        list_values.append(stats)
    return list_values

In [ ]:
def sar_time_series_image(start, end):
    path = "S:\course\geo441\stud\mfeyen\data\S1_vh"
    dir_list = os.listdir(path)
    #print(dir_list)
        
    index_start = dir_list.index(start)
    index_end = dir_list.index(end)
    
    n = len(dir_list[index_start:index_end])
    fig, axs = plt.subplots(n // 2, 2, figsize=(18, (n // 2) * 6))
    im = 0
    ax2 = 0
    
    for filename in dir_list[index_start:index_end]:
        ds = rasterio.open(f"{path}/{filename}")
        
        band1 = ds.read(1)
        band1 = band1[2190:2473, 1093:1475]
        band1 = np.log10(band1)
    
        water_mask = np.zeros((283, 382))
    
        for i in range(len(band1)):
            for j in range(len(band1[i])):
                if band1[i][j] < -2:
                    water_mask[i][j] = 1
                else: water_mask[i][j] = float("nan")
        
        axs[im, ax2].imshow(band1, cmap = "gray")
        axs[im, ax2].imshow(water_mask, cmap = "spring")        
        axs[im, ax2].set_title(filename[12:20], fontsize = 20)
        im += 1
        if im == n // 2:
            ax2 += 1
            im = 0
            
    plt.savefig(f"Irrigation_Period_East{filename[12:16]}")

## Combined
For certain dates, the indices are calculated and showed as masks on their according RGB-image. The SAR-mask is calculated and multiplied with the Landsat 8 - mask, all products are shown on the same RGB image.

In [ ]:
def time_series(list_l8, list_sar, name):
    
    corr = []
    n = len(list_l8)
    fig, axs = plt.subplots(3, n, figsize=(40,18))
    
    names_l8 = []
    for filename in list_l8:
        names_l8.append(filename[10:18])
        
    names_sar = []
    for filename in list_sar:
        names_sar.append(filename[12:20])
    
    for i in range(len(list_l8)):
        rgb = show_RGB(list_l8[i])
        l8_mask = l8_water_mask_NDVI(list_l8[i])        
        sar_mask = sar_water_mask(list_sar[i])
        combined = l8_mask * sar_mask

        axs[0, i].imshow(rgb)
        axs[0, i].imshow(l8_mask, cmap = "spring")        
        axs[0, i].set_title(names_l8[i], fontsize = 24)
        #axs[0, i].axis("off")
        axs[1, i].imshow(rgb)
        axs[1, i].imshow(sar_mask, cmap = "spring")  
        axs[1, i].set_title(names_sar[i], fontsize = 24)
        #axs[1, i].axis("off")
        axs[2, i].imshow(rgb)
        axs[2, i].imshow(combined, cmap = "spring")
        #axs[2, i].axis("off")
        
    axs[0,0].set_ylabel("LSWI - NDVI", fontsize = 24)  
    axs[1,0].set_ylabel("SAR", fontsize = 28)
    axs[2,0].set_ylabel("(LSWI - NDVI) * SAR",fontsize = 28)
    plt.savefig(f"Ricefield1_{name}")
    return corr